# Constants:

In [1]:
import warnings
from sage.symbolic.units import convert
from IPython.display import display
warnings.simplefilter('ignore')

%display latex

# declare units
V = units.electric_potential.volt
A = units.current.amp
ohm = units.resistance.ohm
m = units.length.meter
mm = units.length.millimeter

mt_voltage = 6 * V

# https://en.wikipedia.org/wiki/American_wire_gauge

resistivity = {
    29: 0.2685 * ohm / m,
    30: 0.3386 * ohm / m,
    31: 0.4269 * ohm / m,
    32: 0.5383 * ohm / m,
    33: 0.6788 * ohm / m,
    34: 0.8560 * ohm / m,
    35: 1.079 * ohm / m,
    36: 1.361 * ohm / m,
    37: 1.716 * ohm / m,
    38: 2.164 * ohm / m,
    39: 2.729 * ohm / m,
    40: 3.441 * ohm / m
}

width = {
    29: 0.286 * mm,
    30: 0.254 * mm,
    31: 0.227 * mm,
    32: 0.202 * mm,
    33: 0.180 * mm,
    34: 0.160 * mm,
    35: 0.143 * mm,
    36: 0.127 * mm,
    37: 0.113 * mm,
    38: 0.101 * mm,
    39: 0.0897 * mm,
    40: 0.0799 * mm
}

# Ferrocore Calculations:

In [14]:
l, r, N = var( 'l r N' )
u_r = var( 'u_r', latex_name="\mu_r" ) 
N_d = var( 'N_d', latex_name="N_d" )

ferrocore_gauge = 40
ferrocore_radius = 3.1 * mm
ferrocore_length = 62 * mm
ferrocore_layers = 4
ferrocore_turns = int( ferrocore_length * ferrocore_layers / width[ferrocore_gauge] )

print( ferrocore_turns )

# the length and radius are greater than 0 and the number of turns is more than 1
assume( l > 0 )
assume( r > 0 )
assume( N >= 1 )

# Demagnetization and amplification formulae used from:
# http://www.aerospades.com/uploads/3/7/3/2/37325123/cadre_torquers.pdf
demagnetization( l, r ) = 4 * ( ln( l / r ) - 1 ) / ( ( l / r )^2 - 4 * ln( l / r ) )

amplification( r, l, u_r ) = 1 + ( u_r - 1 ) / ( 1 + ( u_r - 1 ) * demagnetization( l, r ) )

current_ferrocore( r, N ) = mt_voltage / ( 2 * pi * r * N * resistivity[39] )

dipole_ferrocore( r, l, N, u_r ) = pi * r^2 * N * current_ferrocore( N, r ) * amplification( r, l, u_r )

3103


### Ferrocore current at standard parameters:

In [15]:
(current_ferrocore(
    r=ferrocore_radius,
    N=ferrocore_turns ).convert( A ) / A).n(30) * A

0.036376790*amp

### Ferrocore dipole at standard parameters:

In [16]:
# u_r = 15000
# http://www.edfagan.com/hiperco-50-coil.php

dipole_ferrocore( 
    r=ferrocore_radius,
    l=ferrocore_length, 
    N=ferrocore_turns,
    u_r=15000 ).convert( A * m^2 )

0.168513847830963*(amp*meter^2)

# Aircore calculations:

All outputs in A m<sup>2</sup>

In [20]:
l, w, N = var( 'l w N' )

# the length and width are positive, and the number of turns is greater than 1.
# Helps with symbolic optimization and debugging.
assume( l > 0 )
assume( w > 0 )
assume( N >= 1 )

aircore_gauge = 38
aircore_height = 13 * mm
aircore_layers = 50
aircore_width = 67 * mm + ( aircore_layers * width[aircore_gauge] )
aircore_length = 49 * mm + ( aircore_layers * width[aircore_gauge] )
aircore_turns = int( aircore_height / width[aircore_gauge] ) * aircore_layers

print( aircore_turns )

# the current is the minimum between Ohm's law on 32 AWG wire and the max output of 50 mA
current_aircore( l, w, N ) = mt_voltage / ( ( 2 * l + 2 * w ) * N * resistivity[32] )

# Basically, the dipole moment of the aircore is mu = NIA
dipole_aircore( l, w, N ) = N * current_aircore(l, w, N) * (l * w)

6400


### Aircore current at standard parameters:

In [21]:
current_aircore(
    l=aircore_length,
    w=aircore_width,
    N=aircore_turns ).convert( A )

0.00690560629160766*amp

### Aircore dipole at standard parameters:

In [22]:
dipole_aircore(
    l=aircore_length,
    w=aircore_width,
    N=aircore_turns ).convert( A * m^2 )

0.172112127010710*(amp*meter^2)